Os dados estão relacionados com campanhas de marketing direto de uma instituição bancária portuguesa. 
As campanhas de marketing foram baseadas em telefonemas.
Muitas vezes, era necessário mais de um contato com o mesmo cliente, para acessar se o produto (depósito a prazo bancário) seria ('sim') ou não ('não') subscrito.

About this file
Attribute Information:

age (numeric)

job : type of job (categorical:admin.','bluecollar','entrepreneur','housemaid','management','retired','selfemployed','services','student','technician','unemployed','unknown')

marital : marital status (categorical:'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

default: has credit in default? (categorical: 'no','yes','unknown') 

balance: average yearly balance, in euros (numeric)

housing: has housing loan? (categorical: 'no','yes','unknown') Crédito Habitação

loan: has personal loan? (categorical: 'no','yes','unknown')  Empréstimo Pessoal

contact: contact communication type (categorical:'cellular','telephone')

day: last contact day of the month (numeric 1 -31)

month: last contact month of year (categorical: 'jan', 'feb','mar', …, 'nov', 'dec')

duration: last contact duration, in seconds (numeric).
Important note: this attribute highly affects the output target (e.g., ifduration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known.Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic
predictive model.

campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

previous: number of contacts performed before this campaign and for this client (numeric)

poutcome: outcome of the previous marketing campaign(categorical: 'failure','nonexistent','success')

target: has the client subscribed a term deposit? (binary:"yes","no")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('bank-full.csv')
df.head(10).T

In [ ]:
df.shape

In [ ]:
df['Target'].unique()

In [ ]:
# Check data type
# df.dtypes

df.info()

In [ ]:
# Avaliando a quantidade de NA´s (missing)
# df.isnull().sum()
pd.DataFrame(zip(df.isnull().sum(), df.isnull().sum()/len(df)), columns=['Count', 'Proportion'], index=df.columns)


In [ ]:
# check duplicates
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
# definindo uma função com vários aspectos de analise

def resumetable(df):
    print(f"Dataset Shaoe: {df.shape}")
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [ ]:
result =resumetable(df)
result


In [ ]:
# Check how many categorical samples are there in column 'Target'
df['Target'].value_counts()

In [ ]:
# Uma variável é a variável target:  
df.Target.value_counts()  


(df.Target.value_counts() / df.shape[0]) * 100

In [ ]:
# Visualise count plot
sns.countplot(data=df,x='Target')
plt.show()

In [ ]:
# Variável age(idade)  com a variável target  (Objetivo)
import matplotlib.pyplot as plt  
sns.boxplot(x='Target', y='age', data=df)
plt.title('Distribuição de age por classe da variável target')
plt.show()

In [ ]:
# Verificando a variável balance

sns.histplot(data=df, x="balance");

In [ ]:
# Variável balance  com a variável target  (Objetivo)
import matplotlib.pyplot as plt  
sns.boxplot(x='Target', y='balance', data=df)
plt.title('Distribuição de balance por classe da variável target')
plt.show()

In [ ]:
# Transformando a variável "balance" em  categorias

balance_cat = []
for dados in df['balance']:
    if dados <= 0:
        balance_cat.append('0')
    elif dados <= 500:
        balance_cat.append('1')
    elif dados <= 2000:
        balance_cat.append('2')
    else:
        balance_cat.append("3")#atribuindo a nova variavel ao conjunto de dados
df['balance_cat'] = balance_cat
#resultado
df.head().T
# mudar o formato da variável  
df['balance_cat'] =df['balance_cat'].astype(int)

In [ ]:
# comparison between age_cat and target

balance_cat= pd.crosstab(df['Target'],df['balance_cat'])
balance_cat.div(balance_cat.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(6, 6));

In [ ]:
# Variável duration  com a variável target  (Objetivo)
import matplotlib.pyplot as plt  
sns.boxplot(x='Target', y='duration', data=df)
plt.title('Distribuição de duration por classe da variável target')
plt.show()

In [ ]:
# Variável campaign  com a variável target  (Objetivo)
import matplotlib.pyplot as plt  
sns.boxplot(x='Target', y='campaign', data=df)
plt.title('Distribuição campaign por classe da variável target')
plt.show()

In [ ]:
# Variável pdays  com a variável target  (Objetivo)
import matplotlib.pyplot as plt  
sns.boxplot(x='Target', y='pdays', data=df)
plt.title('Distribuição de pdays por classe da variável target')
plt.show()

In [ ]:
# comparison between job  and target

Job= pd.crosstab(df['Target'],df['job'])
Job.div(Job.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(6, 6));

In [ ]:
# comparison between marital  and target

Marital= pd.crosstab(df['Target'],df['marital'])
Marital.div(Job.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(6, 6));

In [ ]:
# comparison between job  and target

Education= pd.crosstab(df['Target'],df['education'])
Education.div(Education.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(6, 6));

https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier

Ensemble

In [ ]:
# Instalando
!pip install catboost

In [ ]:
# importando bibliotecas 
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score


In [ ]:
# Transformando a variável  target

df['y'] = df['Target'].map({'no': 0, 'yes': 1}).astype(int)

df.drop(['Target'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
 
X = df.loc[:, df.columns!='y']
y = df.loc[:, 'y']


In [ ]:
 y

In [ ]:
# seoparando amostra de treino e teste

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.30, random_state=17)

In [ ]:
X_train.head()

In [ ]:
y_train

In [ ]:

cat_features = ['job' , 'marital','education','default', 'housing', 'loan', 'contact',  'month', 'poutcome']


In [ ]:
from catboost import CatBoostClassifier
 
model_catb = CatBoostClassifier(task_type='CPU', iterations =100, random_state=17, eval_metric ='Accuracy', verbose=5)


In [ ]:
# model.fit(X_train, y_train, cat_features=cat_features, plot=True )

model_catb.fit(X_train, y_train, cat_features=cat_features, plot=False , eval_set=(X_test, y_test))

In [ ]:
# Get predicted classes
# preds_class = model_catb.predict(X_test, prediction_type='RawFormulaVal')
preds_class = model_catb.predict(X_test)

In [ ]:
preds_class

In [ ]:
print(model_catb.predict_proba(X_test))

In [ ]:
# Predizendo as probabilidades
yhat = model_catb.predict_proba(X_test) #prediction
yhat

In [ ]:
y_pred_prob = yhat[:, 1] # manter somente para a classe positiva
y_pred_prob

In [ ]:
from sklearn.metrics import confusion_matrix

# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, model_catb.predict(X_test))
confusion_matrix

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model_catb.predict(X_test)))

# Resultado do classification_report:

In [ ]:
#converting probabilities into 0 or 1
for i in range(0,len(X_test)):
      if y_pred_prob[i]>0.3:    #setting threshold to 0.5  e troque para 0.3 (Veja o que acontece com a matriz de confusão!!!)
         y_pred_prob[i]=1
      else:
           y_pred_prob[i]=0

In [ ]:
y_pred_prob

In [ ]:
from sklearn.metrics import confusion_matrix

# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, y_pred_prob)
confusion_matrix

In [ ]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=100, 
    learning_rate=0.8, eval_metric ='Accuracy'
    #loss_function='CrossEntropy'
)


In [ ]:


clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_test, y_test), 
        verbose=10
)


In [ ]:
print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())


In [ ]:
print(clf.predict_proba(X_test))

In [ ]:
print(clf.predict(X_test))

In [ ]:
from sklearn.metrics import confusion_matrix

# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, clf.predict(X_test))
confusion_matrix

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

# Resultado do classification_report:

In [ ]:
clf = CatBoostClassifier(
    iterations=97,
    random_seed=17,
    learning_rate=1,
    custom_loss=['AUC', 'Accuracy']
)

In [ ]:
clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    verbose=False,
    plot=True
)